In [2]:
import sys; sys.path.append('..')
import numpy as np
import torch
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Subset
from cifar10_models.clip_models import ClipViTB32
from src.data_utils import get_dataset, cifar10_label_names
from src.simple_utils import load_pickle
from os.path import join as pjoin
from collections import Counter


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
from src.simple_utils import load_pickle
from src.data_utils import cifar10_label_names, get_dataset
from os.path import join

print("loading data ...")
grouped = load_pickle('../processed/group.pkl')
scores = load_pickle('../processed/scores/scores.pkl')
print("done")


# Test sets in this data
test_sets = ["CIFAR10_test"]
# name of training dataset
train_set = "CIFAR10_train"


print("loading dataset ...")


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_dir = "../ClipViTB32/"

loading data ...
done
loading dataset ...


In [9]:
def test_acc(dataloader, model): 
    acc = []

    for i, batch in enumerate(dataloader):
        if i % 100 ==  0:
            print(f'batch_idx: {i}')

        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)

        with torch.no_grad():
            logits = model(imgs)
            preds = torch.argmax(logits, dim=1)
            acc.append((preds == labels))

    acc = torch.cat(acc)
    acc = acc.cpu().numpy()

    return np.mean(acc)

In [10]:
# %%capture
map_acc = {}
for vit_dir in os.listdir(model_dir): 
    print(f"Model %s" % vit_dir)
    checkpoint_dir = vit_dir
    checkpoint_name = "last.ckpt"
    path = pjoin(model_dir, checkpoint_dir, checkpoint_name)

    checkpoint = torch.load(path)
    state_dict = checkpoint['state_dict']
    state_dict = {k.split('.', 1)[1]: v for k, v in state_dict.items()}

    model = ClipViTB32()
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    dataset = {}
    labels = {}

    for name in test_sets: 
        dataset[name] = get_dataset('../data', dset = name, transform = model.preprocess)
        labels[name] = np.array(dataset[name].targets)
    print("done")
    
    dataloader = DataLoader(dataset[name], batch_size=512, shuffle=False)
    
    acc = test_acc(dataloader, model)
    
    map_acc[vit_dir] = acc 
    print(f"Accuracy %.3f \n\n" % acc)

Model checkpoints13488
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.933 


Model checkpoints83200
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.933 


Model checkpoints78697
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.931 


Model checkpoints18701
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.935 


Model checkpoints81272
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.933 


Model checkpoints13419
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.932 


Model checkpoints4795
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.932 


Model checkpoints49088
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.932 


Model checkpoints90533
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.932 


Model checkpoints53725
Files already downloaded and verified
done
batch_idx: 0
Accuracy 0.933 


Model checkpoints87877
Files al

KeyboardInterrupt: 